In [23]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown,display
import openai

In [ ]:
load_dotenv=("api.env")
api_key=os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No Api key found")
elif api_key[:8]!="sk-proj-":
    print("an api key was found but not open ai")
elif api_key.strip()!=api_key:
    print("found but has spaces")
else:
    print("good to go")

good to go


In [25]:
openai=openai.OpenAI()

In [ ]:
# A class to represent a Webpage
class Website:
    """
    Class to represent website we have scrapped
    """
    url:str
    title:str
    text:str

    def __init__(self,url):
        """
        website from beauitfulSoup"""

        self.url=url
        response=requests.get(url)
        soup=BeautifulSoup(response.content,'html.parser')
        self.title=soup.title.string if soup.title else "No title found"
        for irrelevant in  soup.body(["script","style","img","input"]):
            irrelevant.decompose()
        self.text=soup.body.get_text(separator="\n",strip=True)

In [28]:
# ed=Website("https://www.scirp.org/journal/paperinformation?paperid=31320")
ed=Website("https://en.wikipedia.org/wiki/Large_language_model")
print(ed.title)
print(ed.text)

Large language model - Wikipedia
Jump to content
Main menu
Main menu
move to sidebar
hide
Navigation
Main page
Contents
Current events
Random article
About Wikipedia
Contact us
Contribute
Help
Learn to edit
Community portal
Recent changes
Upload file
Search
Search
Appearance
Donate
Create account
Log in
Personal tools
Donate
Create account
Log in
Pages for logged out editors
learn more
Contributions
Talk
Contents
move to sidebar
hide
(Top)
1
History
2
Dataset preprocessing
Toggle Dataset preprocessing subsection
2.1
Tokenization
2.1.1
BPE
2.1.2
Problems
2.2
Dataset cleaning
2.3
Synthetic data
3
Training and architecture
Toggle Training and architecture subsection
3.1
Reinforcement learning from human feedback (RLHF)
3.2
Instruction tuning
3.3
Mixture of experts
3.4
Prompt engineering, attention mechanism, and context window
3.5
Infrastructure
4
Training cost
5
Tool use
6
Agency
7
Compression
8
Multimodality
9
Properties
Toggle Properties subsection
9.1
Scaling laws
9.2
Emergent abiliti

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [ ]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt="You are an assistant that analyzes the contents of a website \
    and provide a short summary, ignoring text that might be navigation related. \
        Respond in markdown"

In [ ]:
# A function that writes a User Prompt that asks for summaries of websites:
# See how this function creates exactly the format above

def user_prompt_for(website):
    user_prompt=f"You are looking at a website titled {website.title}"
    user_prompt +="The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt +=website.text
    return user_prompt

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

In [ ]:
# See how this function creates exactly the format above
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

## Time to bring it together - the API for OpenAI is very simple!

In [ ]:
# And now: call the OpenAI API. You will get very familiar with this!
def summarize(url):
    website=Website(url)
    response=openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages_for(website)
    )
    return response.choices[0].message.content

In [34]:
summarize("https://en.wikipedia.org/wiki/Large_language_model")

'# Summary of the Wikipedia Page on Large Language Models\n\nThe Wikipedia page for **Large Language Models (LLMs)** provides an extensive overview of their history, architecture, training, evaluation, and implications in the field of Artificial Intelligence, particularly in natural language processing tasks.\n\n## Key Sections:\n\n### 1. **Overview of LLMs**:\n   - LLMs are designed for tasks like language generation, leveraging vast amounts of text to learn statistical relationships.\n   - They typically utilize a transformer architecture for processing and generation.\n\n### 2. **History**:\n   - The development of LLMs has evolved significantly since the 1990s, where early models focused on statistical methods.\n   - Key milestones include the introduction of neural networks to language tasks in the early 2010s and the release of notable models like BERT and GPT series.\n\n### 3. **Dataset Preprocessing**:\n   - Tokenization, dataset cleaning, and the use of synthetic data are cruc

In [ ]:
# A function to display this nicely in the Jupyter output, using markdown
def display_summary(url):
    summary=summarize(url)
    display(Markdown(summary))

In [36]:
display_summary("https://en.wikipedia.org/wiki/Large_language_model")

# Summary of the "Large Language Model" Wikipedia Page

The _Large Language Model_ (LLM) Wikipedia page provides a comprehensive overview of large language models, which are advanced computational models designed specifically for natural language processing (NLP). Here’s a summary of the key sections:

## Key Sections

### 1. Definition and Functionality
LLMs are a type of artificial neural network that performs various tasks related to natural language, such as language generation, by learning from vast datasets. They utilize a transformer-based architecture that processes text efficiently and can be fine-tuned for specific tasks.

### 2. History
The development of LLMs has evolved significantly from earlier statistical models to current architectures, such as transformers introduced in 2017. Key milestones include the introduction of models like BERT, GPT-2, and GPT-3, each building on previous capabilities.

### 3. Dataset Preprocessing
Training LLMs involves several steps:
- **Tokenization**: Converting text into numerical format.
- **Dataset Cleaning**: Removing undesirable data from the training sets.
- **Synthetic Data**: Generating additional training data when necessary.

### 4. Training and Architecture
LLMs are trained using techniques such as reinforcement learning from human feedback (RLHF) and instruction tuning. The architecture generally relies on mechanisms like attention to understand context better.

### 5. Properties and Evaluation
The page discusses various emergent properties of LLMs, including their ability to exhibit behavior not explicitly programmed, and how their performance is evaluated through perplexity and various standard benchmarks.

### 6. Wider Impact
The implementation of LLMs has societal implications, including risks of misinformation, algorithmic bias, and potential industry disruption. The text notes a significant projected impact on global GDP due to generative AI applications.

### 7. List of LLMs
The page includes a list of notable LLMs, detailing their parameters, training costs, and other specifications, demonstrating the diverse landscape of models currently available.

## Conclusion
This Wikipedia page serves as an extensive source of information about large language models, touching on their definitions, historical context, methods of training, evaluation metrics, and broader societal implications. It emphasizes the transformative potential of these models in various fields while also highlighting the challenges and risks associated with their deployment.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [37]:
display_summary("https://www.aajtak.in/")

# Website Summary: Hindi News

The Hindi News website, primarily affiliated with Aaj Tak, provides the latest updates, breaking news, and comprehensive coverage on a variety of topics tailored for Hindi-speaking audiences. It features news on politics, entertainment, sports, business, technology, health, and lifestyle, as well as localized reports from various states in India.

### Key Highlights:
- **Political Coverage**: Extensive reports on upcoming elections, including Maharashtra and Jharkhand, alongside commentary from political leaders. Notable events include Prime Minister Modi’s birthday wishes to L.K. Advani and discussions surrounding the NCP’s use of election materials.
  
- **Entertainment**: Updates include Bollywood news, OTT releases for the weekend, and features on popular TV shows and celebrities.

- **Sports Section**: Covers live cricket updates, including India's matches, and provides insights into upcoming tournaments like the IPL.

- **Health and Lifestyle**: Articles on wellness tips, dietary advice, and lifestyle trends that promote better health and fitness.

- **Technology and Business**: This section discusses business trends, stock market updates, and technological advancements relevant to readers.

- **Cultural Insights**: Focuses on festivals, religious observances, and astrology, offering readers guidance on various customs and practices.

### Recent Announcements:
- Prime Minister Modi visited L.K. Advani to wish him a happy birthday.
- There’s a significant focus on the Jharkhand and Maharashtra elections with various political personalities making headlines.

Overall, the Hindi News website serves as a comprehensive platform for Hindi-speaking individuals seeking updated news, cultural content, and diverse information across multiple sectors.

## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them.